<a href="https://colab.research.google.com/github/Danielb711/Xml-Roberta-POS-Tagging/blob/main/POS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Librerías necesarias**

In [3]:
#!pip install conllu
#!pip install transformers
#!pip install torch
#!pip install seqeval
#!pip install huggingface_hub

from google.colab import drive
import os
from conllu import parse
from transformers import XLMRobertaTokenizerFast, XLMRobertaForTokenClassification, AdamW
import torch
from torch.utils.data import Dataset, DataLoader
from torch.nn import CrossEntropyLoss
import numpy as np
from sklearn.metrics import classification_report
from huggingface_hub import notebook_login
from huggingface_hub import HfApi, upload_folder

# **Acceso a los datos**

## Montamos el drive

In [4]:
# Montamos el drive para acceder a los archivos
drive.mount('/content/drive')

Mounted at /content/drive


## Lista de archivos

In [5]:
# Ruta a la carpeta dentro de Google Drive
folder_path = '/content/drive/My Drive/ServicioSocial/UniversalDependencies/UD_Spanish-AnCora'

# Listar los archivos en la carpeta
files = os.listdir(folder_path)
print("Archivos en la carpeta:", files)

Archivos en la carpeta: ['README.md', 'LICENSE.txt', 'es_ancora-ud-dev.conllu', 'es_ancora-ud-test.conllu', 'es_ancora-ud-train.conllu', 'stats.xml', 'es_ancora-ud-dev.txt', 'es_ancora-ud-test.txt', 'es_ancora-ud-train.txt']


## **Datos de entrenamiento**

In [6]:
# Accedemos a los datos de entrenamiento
file_path = os.path.join(folder_path, 'es_ancora-ud-train.conllu')

# Abrir y leer el contenido del archivo
with open(file_path, 'r') as file:
    entrenamiento = file.read()

# Muestra las primeras 500 caracteres para verificar el contenido
print(entrenamiento[:500])

# global.Entity = eid-etype-head-other
# newdoc id = 3LB-CAST-104_c-1
# sent_id = 3LB-CAST-104_c-1-s1
# text = Las reservas de oro y divisas de Rusia subieron 800 millones de dólares y el 26 de mayo equivalían a 19.100 millones de dólares, informó hoy un comunicado del Banco Central.
# orig_file_sentence 001#58
1	Las	el	DET	da0fp0	Definite=Def|Gender=Fem|Number=Plur|PronType=Art	2	det	2:det	_
2	reservas	reserva	NOUN	ncfp000	Gender=Fem|Number=Plur	9	nsubj	9:nsubj	ArgTem=arg1:tem
3	de	de	ADP	sps00


### Formato ConLLu

In [7]:
# Parsea el contenido del archivo
sentences_train = parse(entrenamiento)

### Como se ve el contenido

In [8]:
sentences_train[0]

TokenList<Las, reservas, de, oro, y, divisas, de, Rusia, subieron, 800, millones, de, dólares, y, el, 26, de, mayo, equivalían, a, 19.100, millones, de, dólares, ,, informó, hoy, un, comunicado, del, de, el, Banco, Central, ., metadata={global.Entity: "eid-etype-head-other", newdoc id: "3LB-CAST-104_c-1", sent_id: "3LB-CAST-104_c-1-s1", text: "Las reservas de oro y divisas de Rusia subieron 800 millones de dólares y el 26 de mayo equivalían a 19.100 millones de dólares, informó hoy un comunicado del Banco Central."}>

In [9]:
sentences_train[0][0]

{'id': 1,
 'form': 'Las',
 'lemma': 'el',
 'upos': 'DET',
 'xpos': 'da0fp0',
 'feats': {'Definite': 'Def',
  'Gender': 'Fem',
  'Number': 'Plur',
  'PronType': 'Art'},
 'head': 2,
 'deprel': 'det',
 'deps': [('det', 2)],
 'misc': None}

### Obtenemos los datos que nos interesan para pos tagging que es la palabra y su POS

In [10]:
oraciones_train = []
labels_train = []

for sentence in sentences_train:
    palabras = [token['form'] for token in sentence]
    pos_tags = [token['upos'] for token in sentence]

    oraciones_train.append(palabras)
    labels_train.append(pos_tags)

## **Datos de validación**

In [11]:
# Accedemos a los datos de validación
file_path = os.path.join(folder_path, 'es_ancora-ud-dev.conllu')

# Abrir y leer el contenido del archivo
with open(file_path, 'r') as file:
    validacion = file.read()

# Muestra las primeras 500 caracteres para verificar el contenido
print(validacion[:500])

# global.Entity = eid-etype-head-other
# newdoc id = 3LB-CAST-111_C-2
# sent_id = 3LB-CAST-111_C-2-s1
# text = El gobernante, con ganada fama desde que llegó hace 16 meses al poder de explotar al máximo su oratoria y acusado por sus detractores de incontinencia verbal, enmudeció desde el momento en el que el Tribunal Supremo de Justicia (TSJ) decidió suspender temporalmente los comicios múltiples ante la imposibilidad "técnica" de celebrarlos el 28 de mayo.
# orig_file_sentence 001#1
1	El	el	DET


### Formato ConLLu

In [12]:
# Parsea el contenido del archivo
sentences_val = parse(validacion)

### Como se ve el contenido

In [13]:
sentences_val[0]

TokenList<El, gobernante, ,, con, ganada, fama, desde, que, llegó, hace, 16, meses, al, a, el, poder, de, explotar, al, a, el, máximo, su, oratoria, y, acusado, por, sus, detractores, de, incontinencia, verbal, ,, enmudeció, desde, el, momento, en, el, que, el, Tribunal, Supremo, de, Justicia, (, TSJ, ), decidió, suspender, temporalmente, los, comicios, múltiples, ante, la, imposibilidad, ", técnica, ", de, celebrarlos, celebrar, los, el, 28, de, mayo, ., metadata={global.Entity: "eid-etype-head-other", newdoc id: "3LB-CAST-111_C-2", sent_id: "3LB-CAST-111_C-2-s1", text: "El gobernante, con ganada fama desde que llegó hace 16 meses al poder de explotar al máximo su oratoria y acusado por sus detractores de incontinencia verbal, enmudeció desde el momento en el que el Tribunal Supremo de Justicia (TSJ) decidió suspender temporalmente los comicios múltiples ante la imposibilidad "técnica" de celebrarlos el 28 de mayo."}>

In [14]:
sentences_val[0][0]

{'id': 1,
 'form': 'El',
 'lemma': 'el',
 'upos': 'DET',
 'xpos': 'da0ms0',
 'feats': {'Definite': 'Def',
  'Gender': 'Masc',
  'Number': 'Sing',
  'PronType': 'Art'},
 'head': 2,
 'deprel': 'det',
 'deps': [('det', 2)],
 'misc': None}

### Obtenemos los datos que nos interesan para pos tagging que es la palabra y su POS

In [15]:
oraciones_val = []
labels_val = []

for sentence in sentences_val:
    palabras = [token['form'] for token in sentence]
    pos_tags = [token['upos'] for token in sentence]

    oraciones_val.append(palabras)
    labels_val.append(pos_tags)

## **Datos de Testeo**

In [16]:
# Accedemos a los datos de validación
file_path = os.path.join(folder_path, 'es_ancora-ud-test.conllu')

# Abrir y leer el contenido del archivo
with open(file_path, 'r') as file:
    test = file.read()

# Muestra las primeras 500 caracteres para verificar el contenido
print(test[:500])

# global.Entity = eid-etype-head-other
# newdoc id = 3LB-CAST-111_C-4
# sent_id = 3LB-CAST-111_C-4-s1
# text = Partidario de la "perestroika" de Mijail Gorbachov en la Unión Soviética, en 1989 entró en conflicto con Yívkov, líder durante 35 años del Partido Comunista y del Estado búlgaro, y le acusó en una carta abierta de utilizar métodos poco democráticos de gobierno.
# orig_file_sentence 001#1
1	Partidario	partidario	ADJ	aq0ms0	Gender=Masc|Number=Sing	17	amod	17:amod	_
2	de	de	ADP	sps00	_	5	c


### Formato ConLLu

In [17]:
# Parsea el contenido del archivo
sentences_test = parse(test)

### Como se ve el contenido

In [18]:
sentences_test[0]

TokenList<Partidario, de, la, ", perestroika, ", de, Mijail, Gorbachov, en, la, Unión, Soviética, ,, en, 1989, entró, en, conflicto, con, Yívkov, ,, líder, durante, 35, años, del, de, el, Partido, Comunista, y, del, de, el, Estado, búlgaro, ,, y, le, acusó, en, una, carta, abierta, de, utilizar, métodos, poco, democráticos, de, gobierno, ., metadata={global.Entity: "eid-etype-head-other", newdoc id: "3LB-CAST-111_C-4", sent_id: "3LB-CAST-111_C-4-s1", text: "Partidario de la "perestroika" de Mijail Gorbachov en la Unión Soviética, en 1989 entró en conflicto con Yívkov, líder durante 35 años del Partido Comunista y del Estado búlgaro, y le acusó en una carta abierta de utilizar métodos poco democráticos de gobierno."}>

In [19]:
sentences_test[0][0]

{'id': 1,
 'form': 'Partidario',
 'lemma': 'partidario',
 'upos': 'ADJ',
 'xpos': 'aq0ms0',
 'feats': {'Gender': 'Masc', 'Number': 'Sing'},
 'head': 17,
 'deprel': 'amod',
 'deps': [('amod', 17)],
 'misc': None}

### Obtenemos los datos que nos interesan para pos tagging que es la palabra y su POS

In [20]:
oraciones_test = []
labels_test = []

for sentence in sentences_test:
    palabras = [token['form'] for token in sentence]
    pos_tags = [token['upos'] for token in sentence]

    oraciones_test.append(palabras)
    labels_test.append(pos_tags)

# **Modelado con Transformers**

## **Tokenizador**

Antes de proceder con el entrenamiento o la evaluación de nuestro modelo, necesitamos preparar nuestros datos de texto para que estén en un formato adecuado que el modelo pueda entender. Esto implica convertir el texto crudo en una secuencia de tokens o unidades de texto más pequeñas. Para realizar esta tarea, utilizamos un tokenizador.

El tokenizador desempeña varias funciones clave:

* Divide el texto: Transforma el texto crudo en tokens, que pueden ser palabras,
partes de palabras, o incluso caracteres, dependiendo del vocabulario preentrenado del modelo.
* Convierte a IDs: Asigna a cada token un identificador único según el vocabulario del modelo preentrenado.
* Añade tokens especiales: Inserta tokens especiales requeridos por el modelo para entender correctamente la estructura del texto, como los tokens de inicio, separación y relleno.
* Crea máscaras de atención: Genera máscaras para indicar al modelo qué partes de la secuencia son tokens reales y cuáles son relleno.

Estos pasos son cruciales para asegurar que el modelo procese el texto de manera efectiva y genere predicciones precisas. Para nuestro modelo XLM-RoBERTa, utilizaremos el XLMRobertaTokenizerFast, el cual está específicamente diseñado para trabajar con el modelo XLM-RoBERTa y optimizado para ofrecer una tokenización rápida y eficiente.

In [21]:
# Cargar el tokenizador
tokenizer = XLMRobertaTokenizerFast.from_pretrained('xlm-roberta-base')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

In [22]:
# Tokenizamos
tokenized_train = tokenizer(oraciones_train, truncation=True, padding=True, is_split_into_words=True, return_tensors="pt")
tokenized_val = tokenizer(oraciones_val, truncation=True, padding=True, is_split_into_words=True, return_tensors="pt")
tokenized_test = tokenizer(oraciones_test, truncation=True, padding=True, is_split_into_words=True, return_tensors="pt")

In [23]:
tokenized_train

{'input_ids': tensor([[     0,   5599,  18433,  ...,      1,      1,      1],
        [     0,  64933,     88,  ...,      1,      1,      1],
        [     0,   3731,  99867,  ...,      1,      1,      1],
        ...,
        [     0,     44, 176684,  ...,      1,      1,      1],
        [     0,     44,    990,  ...,      1,      1,      1],
        [     0,   4228, 178831,  ...,      1,      1,      1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}

## **Alineación de las etiquetas**

Puesto que al tokenizar las palabras puede que estas sean divididas en palabras mas pequeñas si son desconocidas pra el modelo, hay que realinear las etiquetas que tenian asignadas.

### Diccionario que mapea las etiquetas a un id único

In [24]:
unique_labels = set(label for sublist in labels_train for label in sublist)

# Crear el mapeo de etiquetas a ID
label_to_id = {label: id for id, label in enumerate(unique_labels)}

In [25]:
label_to_id

{'CCONJ': 0,
 'NOUN': 1,
 'VERB': 2,
 'AUX': 3,
 'DET': 4,
 'SCONJ': 5,
 'PART': 6,
 'SYM': 7,
 'X': 8,
 'ADV': 9,
 'PRON': 10,
 'PROPN': 11,
 'ADP': 12,
 'PUNCT': 13,
 'NUM': 14,
 '_': 15,
 'ADJ': 16,
 'INTJ': 17}

### **Asignación de POS tags a los nuevos tokens**

Asignamos los ids con base en nuestro diccionarios de Tags, o en su defecto en caso de palabras que fueron descompuestas, solo al primer componente y al segundo asignamos -100, lo cual es una convención de HugginFace para ser ignoradas por el modelo

#### **Entrenamiento**

In [26]:
## nueva lista de labels
labels_train_aligned = []

## Iteramos sobre la lista original de labels
for i, label_seq in enumerate(labels_train):

    # Obtenemos los word_ids para la secuencia tokenizada actual
    word_ids = tokenized_train.word_ids(batch_index=i)

    label_ids = []

    previous_word_id = None

    for word_id in word_ids:

        # Si el word_id es None (token especial) o es un subtoken (mismo word_id que el anterior), asigna -100
        if word_id is None or word_id == previous_word_id:
            label_ids.append(-100)
        else:

            # Obtenemos el id del Tag que correspondia a esa palabra
            label_ids.append(label_to_id[label_seq[word_id]])

        previous_word_id = word_id

    labels_train_aligned.append(label_ids)

In [27]:
## Conversión a tensores
labels_train_tensor = torch.tensor(labels_train_aligned)

## Agregamos las labels al diccionario donde tendremos los dos tensores:
tokenized_train["labels"] = labels_train_tensor

#### **Validación**

In [28]:
## nueva lista de labels
labels_val_aligned = []

## Iteramos sobre la lista original de labels
for i, label_seq in enumerate(labels_val):

    # Obtenemos los word_ids para la secuencia tokenizada actual
    word_ids = tokenized_val.word_ids(batch_index=i)

    label_ids = []

    previous_word_id = None

    for word_id in word_ids:

        # Si el word_id es None (token especial) o es un subtoken (mismo word_id que el anterior), asigna -100
        if word_id is None or word_id == previous_word_id:
            label_ids.append(-100)
        else:

            # Obtenemos el id del Tag que correspondia a esa palabra
            label_ids.append(label_to_id[label_seq[word_id]])

        previous_word_id = word_id

    labels_val_aligned.append(label_ids)

In [29]:
## Conversión a tensores
labels_val_tensor = torch.tensor(labels_val_aligned)

## Agregamos las labels al diccionario donde tendremos los dos tensores:
tokenized_val["labels"] = labels_val_tensor

#### **Testeo**

In [30]:
## nueva lista de labels
labels_test_aligned = []

## Iteramos sobre la lista original de labels
for i, label_seq in enumerate(labels_test):

    # Obtenemos los word_ids para la secuencia tokenizada actual
    word_ids = tokenized_test.word_ids(batch_index=i)

    label_ids = []

    previous_word_id = None

    for word_id in word_ids:

        # Si el word_id es None (token especial) o es un subtoken (mismo word_id que el anterior), asigna -100
        if word_id is None or word_id == previous_word_id:
            label_ids.append(-100)
        else:

            # Obtenemos el id del Tag que correspondia a esa palabra
            label_ids.append(label_to_id[label_seq[word_id]])

        previous_word_id = word_id

    labels_test_aligned.append(label_ids)

In [31]:
## Conversión a tensores
labels_test_tensor = torch.tensor(labels_test_aligned)

## Agregamos las labels al diccionario donde tendremos los dos tensores:
tokenized_test["labels"] = labels_test_tensor

## **Modelo**

Instanciamos el modelo de la clase XLMRobertaForTokenClassification, la cual añade una capa a un modelo preentrenado con la función from_pretained, en este caso 'xml-roberta-base', y en esta capa se realiza el fine tuning para nuestro POS tagging

In [32]:
model = XLMRobertaForTokenClassification.from_pretrained('xlm-roberta-base', num_labels=len(unique_labels))

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForTokenClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## **Entrenamiento**

### **Construcción de DataSets de PyTorch**

La utilización de Datasets personalizados en PyTorch es una práctica clave en el flujo de trabajo de aprendizaje profundo, ofreciendo múltiples ventajas para el manejo de datos complejos y específicos de una tarea, como el etiquetado de parte del discurso (POS tagging) en procesamiento de lenguaje natural (PLN):

* Estandarización de la Entrada de Datos: Permite estandarizar el formato de los datos de entrada y las etiquetas, asegurando que el modelo reciba la información en la estructura esperada y en tipos de datos consistentes, lo cual es crucial para modelos preentrenados como XLM-RoBERTa.

* Escalabilidad y Flexibilidad: Facilita el manejo de grandes volúmenes de datos al cargarlos eficientemente en memoria. PyTorch optimiza el acceso y la manipulación de los datos durante el entrenamiento, lo que permite escalar el proceso a conjuntos de datos más grandes y complejos sin sacrificar el rendimiento.

* Personalización para Preprocesamiento Específico: Cada tarea de PLN puede requerir un preprocesamiento de datos único (p.ej., tokenización, alineación de etiquetas). Los Datasets personalizados permiten integrar estos pasos específicos directamente en el flujo de datos, asegurando que todas las instancias se procesen de manera uniforme antes del entrenamiento.

*Integración con DataLoader de PyTorch: Al utilizar Datasets personalizados en conjunto con DataLoader, se facilita el batching, el muestreo aleatorio y la paralelización en la carga de datos, mejorando la eficiencia del entrenamiento y la evaluación del modelo.

In [33]:
# Definimos nuestra clase heredada de Dataset
class POSDataset(Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

#### **Creamos los Datasets**

In [34]:
train_dataset = POSDataset(tokenized_train)
val_dataset = POSDataset(tokenized_val)
test_dataset = POSDataset(tokenized_test)

### **Carga y Manejo de Datos con DataLoader**

La clase DataLoader de PyTorch es una herramienta esencial para la carga y manejo eficiente de datos durante el entrenamiento y validación de modelos de aprendizaje profundo. Proporciona una interfaz flexible y potente para automatizar la carga de datos en lotes, el barajado de los datos para el entrenamiento, y el aprovechamiento de la carga de datos en paralelo a través de múltiples subprocesos. Su compatibilidad con cualquier clase que herede de torch.utils.data.Dataset lo hace indispensable para trabajar con datasets personalizados, facilitando un entrenamiento eficaz y escalable.

Características Clave:

* Carga en Lotes: Permite el procesamiento de datos en pequeños grupos, haciéndolo manejable y eficiente.
* Barajado de Datos: Mejora la generalización del modelo al presentar los datos en un orden diferente cada época.
* Carga Paralela: Utiliza múltiples subprocesos para cargar datos, acelerando la preparación antes del entrenamiento.

In [35]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=2)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False, num_workers=2)

### **Configuración del Optimizador y Función de Pérdida**

Para entrenar efectivamente nuestro modelo de etiquetado de parte del discurso (POS tagging) con la biblioteca Transformers y PyTorch, necesitamos definir dos componentes críticos: el optimizador y la función de pérdida. Estos elementos son esenciales para guiar el proceso de aprendizaje del modelo, permitiéndonos ajustar sus parámetros internos (pesos) basados en el rendimiento observado durante el entrenamiento.

**Optimizador**

El optimizador es responsable de actualizar los parámetros del modelo en dirección a minimizar la función de pérdida. Una elección común para muchas tareas de procesamiento de lenguaje natural, incluido el etiquetado de parte del discurso, es AdamW. AdamW es una variante del optimizador Adam que incorpora correcciones de decaimiento de peso, lo que ayuda a prevenir el sobreajuste y mejora la generalización en comparación con el Adam estándar.

**Función de Pérdida**

La función de pérdida mide la discrepancia entre las predicciones del modelo y las etiquetas reales. Para tareas de clasificación, como el etiquetado de parte del discurso, una elección común es la Cross-Entropy Loss, que es efectiva para comparar la distribución de las predicciones del modelo con la distribución real de las etiquetas.

In [36]:
# Inicializamos el optimizador AdamW con los parámetros del modelo
optimizer = AdamW(model.parameters(), lr=5e-5)

# Definimos la función de pérdida
loss_fn = CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


### **Early Stopping**

El early stopping es una técnica utilizada para prevenir el sobreajuste durante el entrenamiento de modelos de aprendizaje profundo. La idea detrás de esta técnica es monitorear el rendimiento del modelo en un conjunto de datos de validación y detener el entrenamiento cuando dicho rendimiento deja de mejorar.

Para implementar early stopping, se utilizan tres parámetros clave:

* **patience**: Este parámetro define el número de épocas que esperaremos para observar una mejora en la pérdida de validación antes de detener el entrenamiento. En este caso, se ha establecido una patience de 2, lo que significa que si después de dos épocas consecutivas no se observa una mejora, el entrenamiento se detendrá.

* **best_val_loss**: Representa la mejor pérdida de validación observada hasta el momento. Se inicializa con un valor infinito para asegurar que cualquier pérdida calculada en las primeras épocas se considere como una mejora. Este valor se actualizará a lo largo del entrenamiento cada vez que encontremos una pérdida de validación menor.

* **epochs_no_improve**: Es un contador que lleva la cuenta de cuántas épocas consecutivas han pasado sin observar una mejora en la pérdida de validación. Si este contador alcanza el valor especificado por patience, se activará el early stopping.

In [37]:
# Early stopping
patience = 2  # Número de épocas a esperar después de una mejora antes de detener el entrenamiento
best_val_loss = float('inf')  # Mejor pérdida de validación observada, inicializada con infinito
epochs_no_improve = 0  # Contador de épocas sin mejora

### **Entrenamiento** utilizando los dataloaders, el optimizador y la función de pérdida

In [38]:
# Verificamos si el entorno tiene GPU, y nos aseguramos que el modelo este cargado
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Guardado del mejor modelo
model_save_path = '/content/best_model.pt'

# Modelo en modo entrenamiento
model.train()

# Número de épocas
num_epochs = 8

# Recorremos el conjunto de entrenamiento, la cantidad de épocas definidas
for epoch in range(num_epochs):

    # Pérdida de la época
    total_loss = 0

    # Con nuestros DataLoaders, recorremos los lotes
    for batch in train_loader:

      # Reseteamos los gradientes del optimizador en cada lote, para solo tomar en cuenta el acumulado en la iteración del lote actual
        optimizer.zero_grad()

        # Nos aseguremos que todos nuestros datos se carguen en memoria en el mismo dispositivo
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Obtenemos las predicciones del modelo en el conjunto de entrenamiento y su pérdida
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        total_loss += loss.item()

        # Retropropagación de la pérdida y calculo del gradiente
        loss.backward()

        # Recalibración de los pesos del modelo
        optimizer.step()

    # Pérdida promedio por época respecto a los lotes
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Training Loss: {avg_train_loss}")

    # Validación

    # Modelo en modo evaluación
    model.eval()
    total_eval_loss = 0
    for batch in val_loader:
        with torch.no_grad():
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_eval_loss += loss.item()

    avg_val_loss = total_eval_loss / len(val_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Validation Loss: {avg_val_loss}")

    # Early Stopping
    if avg_val_loss < best_val_loss:
        print(f"Validation loss improved from {best_val_loss} to {avg_val_loss}. Saving best model to {model_save_path}.")
        best_val_loss = avg_val_loss

        epochs_no_improve = 0

        # Guarda el modelo
        torch.save(model.state_dict(), model_save_path)

    else:
        epochs_no_improve += 1
        print(f"Validation loss did not improve. ({epochs_no_improve}/{patience} epochs without improvement)")

    # Verifica si se alcanzó el límite de paciencia para el early stopping
    if epochs_no_improve == patience:
        print("Early stopping triggered.")
        break

    # Vuelve a poner el modelo en modo de entrenamiento para la siguiente época
    model.train()

Epoch 1/8, Training Loss: 0.10896400426183843
Epoch 1/8, Validation Loss: 0.03686694645036299
Validation loss improved from inf to 0.03686694645036299. Saving best model to /content/best_model.pt.
Epoch 2/8, Training Loss: 0.030986487880570256
Epoch 2/8, Validation Loss: 0.03443904785224451
Validation loss improved from 0.03686694645036299 to 0.03443904785224451. Saving best model to /content/best_model.pt.
Epoch 3/8, Training Loss: 0.023318579213428836
Epoch 3/8, Validation Loss: 0.03454647774029022
Validation loss did not improve. (1/2 epochs without improvement)
Epoch 4/8, Training Loss: 0.019904673157529938
Epoch 4/8, Validation Loss: 0.03818064221951108
Validation loss did not improve. (2/2 epochs without improvement)
Early stopping triggered.


## **Evaluación**

El modelo con el mejor rendimiento en el conjunto de validación es evaluado en el conjunto de prueba. Este proceso se enfoca en medir la capacidad del modelo para generalizar a nuevos datos. Los pasos incluyen:

* **Carga del Mejor Modelo**: Se carga el modelo con los mejores parámetros, guardados durante el entrenamiento, para asegurar que la evaluación se realice con la versión más óptima del modelo.

* **Predicciones y Métricas**: El modelo se utiliza para hacer predicciones sobre el conjunto de prueba, y se calculan métricas detalladas como precisión, recall y F1-score para cada etiqueta de POS. Esto proporciona una visión integral del rendimiento y las áreas de fortaleza o debilidad del modelo.

### Carga del mejor modelo

In [39]:
# Cargamos los pesos guardados
model.load_state_dict(torch.load('/content/best_model.pt'))

# Modo evaluación
model.eval()

# Aseguramos la carga en el dsipositivo adecuado
model.to(device)

XLMRobertaForTokenClassification(
  (roberta): XLMRobertaModel(
    (embeddings): XLMRobertaEmbeddings(
      (word_embeddings): Embedding(250002, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): XLMRobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x XLMRobertaLayer(
          (attention): XLMRobertaAttention(
            (self): XLMRobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): XLMRobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bi

### Predicciones

In [40]:
# Inicializamos listas para guardar las predicciones y las etiquetas verdaderas
predictions = []

# Desactivamos el cálculo de gradientes para ahorrar memoria y acelerar la evaluación
with torch.no_grad():
    for batch in test_loader:  # Iteramos sobre el conjunto de testeo

        # Movemos los datos al dispositivo adecuado (GPU o CPU)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Realizamos las predicciones con el modelo
        outputs = model(input_ids, attention_mask=attention_mask)

        # Obtenemos los logits y los movemos al CPU
        logits = outputs.logits.detach().cpu().numpy()
        # Movemos las etiquetas verdaderas al CPU para compararlas con las predicciones
        label_ids = labels.to('cpu').numpy()

        # Guardamos las predicciones y las verdaderas etiquetas
        # Usamos np.argmax para convertir los logits a índices de la etiqueta predicha
        predictions.extend([list(p) for p in np.argmax(logits, axis=-1)])

# En este punto, `predictions` contiene las etiquetas predichas para cada token

### Obtenemos la lista de etiquetas y predicciones en una forma más digerible

In [41]:
# Revertir el mapeo de IDs a etiquetas originales
id_to_label = {id: label for label, id in label_to_id.items()}

# Listas para almacenar las etiquetas finales por oración
final_pred_labels_per_sentence = []

# Reevertimos la tokenización obteniendo los tags correspondientes unicamente a las palabras iniciales.
for i, pred in enumerate(predictions):

    word_ids = tokenized_test.word_ids(batch_index=i)

    sentence_pred_labels = []

    prev_word_id = None

    for word_idx, pred_label in zip(word_ids, pred):

        if word_idx is not None and word_idx != prev_word_id:
            # Solo considera la etiqueta del primer sub-token
            sentence_pred_labels.append(id_to_label[pred_label])
        prev_word_id = word_idx

    final_pred_labels_per_sentence.append(sentence_pred_labels)

### Métricas de evaluación (Seqeval)

In [42]:
# Aplanar las listas de listas
flat_true_labels = [label for sublist in labels_test for label in sublist]
flat_pred_labels = [label for sublist in final_pred_labels_per_sentence for label in sublist]

# Crear un informe de clasificación usando sklearn
report = classification_report(flat_true_labels, flat_pred_labels, zero_division=0)
print(report)

              precision    recall  f1-score   support

         ADJ       0.98      0.96      0.97      3468
         ADP       1.00      1.00      1.00      8332
         ADV       0.98      0.98      0.98      1763
         AUX       0.99      0.98      0.99      1304
       CCONJ       1.00      1.00      1.00      1439
         DET       1.00      1.00      1.00      8055
        INTJ       0.77      0.62      0.69        16
        NOUN       0.98      0.99      0.98      9531
         NUM       0.98      0.97      0.97       977
        PART       0.93      0.78      0.85        18
        PRON       0.99      0.99      0.99      3250
       PROPN       0.99      0.99      0.99      4094
       PUNCT       1.00      1.00      1.00      6334
       SCONJ       0.97      0.98      0.98      1210
         SYM       0.88      0.75      0.81        28
        VERB       0.99      1.00      0.99      4636
           _       1.00      1.00      1.00      1166

    accuracy              